# SWB analysis pipeline part 4: 

Freq-Band Averaged Time Courses - Single ROI

*Created: 07/14/24* \
*Updated: 09/26/2024*


In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.mixed_linear_model import MixedLM 
from joblib import Parallel, delayed
import pickle
import itertools
import time 
from matplotlib.ticker import StrMethodFormatter
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib as mpl
from matplotlib import cm



import warnings
warnings.filterwarnings('ignore')




In [2]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir   = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir  = f'{base_dir}ephys_analysis/behav/data/'

script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

09262024


In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'{base_dir}ephys_analysis/LFPAnalysis/')

from LFPAnalysis import analysis_utils

sys.path.append(f'{script_dir}analysis_notebooks/')

from ieeg_tools import *

sys.path.append(f'{script_dir}behav/')

from behav_utils import *
from swb_subj_behav import *

In [4]:
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
# subj_ids


In [5]:
bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df



,subj_id,bdi
0,MS002,14
1,MS003,8
2,MS009,16
3,MS011,13
4,MS015,26
5,MS016,10
6,MS017,26
7,MS019,12
8,MS022,10
9,MS024,16


In [6]:
### load ROI reref master 
roi_reref_labels_master_df = pd.read_csv(glob(f'{base_dir}ephys_analysis/results/roi_info/roi_reref_labels_master.csv')[0])
roi_reref_labels_master_df = roi_reref_labels_master_df.drop(columns=['Unnamed: 0'])
roi_reref_labels_master_df



,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


# Get data 

In [7]:
freqs = {'hfa':np.array([70,200]),'gamma':np.array([30,70]),'beta':np.array([13,30]),
         'alpha':np.array([8,13]),'theta':np.array([4,8])}

In [20]:
subj_id = 'MS002'
power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5').crop(0,2.0)


Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...


In [21]:
power_epochs._data.shape

(150, 94, 30, 1001)

In [17]:
dir(epoch_test)

['__class__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allow_empty',
 '_annotations',
 '_bad_dropped',
 '_channel_type_idx',
 '_check_consistency',
 '_check_metadata',
 '_compute_aggregate',
 '_current',
 '_data',
 '_data_sel_copy_scale',
 '_decim',
 '_decim_slice',
 '_detrend_offset_decim',
 '_detrend_picks',
 '_do_baseline',
 '_do_delayed_proj',
 '_evoked_from_epoch_data',
 '_filename',
 '_get_channel_positions',
 '_get_data',
 '_get_epoch_from_raw',
 '_get_name',
 '_getitem',
 '_handle_empty',
 '_handle_tmin_tmax',
 '_is_good_epoch',
 '_item_to_select',
 '_keys_to_idx

In [ ]:
### plot_roi = roi str according to roi_reref_labels_master
plot_roi   = 'motor'
### plot_band = str id of band to plot
plot_band  = 'beta'
### freq_range = np array of upper/lower freq bounds of plot_band - either manual or subset from freqs above
# freq_range = np.array([9,12])
freq_range = freqs[plot_band]

In [ ]:
roi_df = roi_reref_labels_master_df[roi_reref_labels_master_df.roi == plot_roi]
roi_df['unique_reref_ch'] = roi_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)
roi_subjs  = roi_df.subj_id.unique().tolist()
roi_elecs  = {f'{subj}':roi_df[roi_df.subj_id == subj].unique_reref_ch.tolist() for subj in roi_subjs}
roi_elecs  = {f'{subj}':roi_df[roi_df.subj_id == subj].reref_ch_names.tolist() for subj in roi_subjs}

roi_elecs

In [ ]:
roi_band_pow = {}
roi_metadata = {}

for subj_id in roi_subjs:
    
    # load cpe power epochs for single subject 
    power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0]
    roi_epochs   = power_epochs.copy().pick_channels(roi_elecs[subj_id])
    del power_epochs
    
    roi_pow_df  = roi_epochs.to_data_frame()
    roi_band_df = roi_pow_df[(roi_pow_df.freq >= freq_range[0])&(roi_pow_df.freq <= freq_range[1])].groupby(
                  ['epoch','time']).agg('mean').reset_index()
    
    roi_band_df  = roi_band_df.melt(id_vars=['epoch','time','freq'],
                                    value_vars=roi_elecs[subj_id],
                                    var_name='reref_ch_names', value_name='band_pow',
                                    ignore_index = False)
    
    roi_band_pow[subj_id] = roi_band_df
    roi_metadata[subj_id] = roi_epochs.metadata.copy()   
    del roi_epochs, roi_pow_df, roi_band_df 
    
     

    

In [ ]:
##### regressors to zscore for plot data 
continuous_regressors = ['decisionCPE','TrialEV','TotalProfit','decisionRegret','decisionRelief','RPE','Profit']


In [ ]:
plot_dfs = []

### update metadata - zscore continuous regressors + add RT info
for subj_id in roi_metadata.keys():
    metadata_df          = roi_metadata[subj_id].reset_index()
    metadata_df['epoch'] = metadata_df.Round-1
    
    roi_df      = roi_band_pow[subj_id]
    roi_df      = roi_df.set_index('epoch')
    
    # merge pow and metadata dfs keeping epoch as index 
    roi_regression_df = pd.merge(roi_df, metadata_df,left_on=['epoch'],
                                right_on=['epoch']).reset_index(drop=True).set_index('epoch')
    
    roi_regression_df  = roi_regression_df[roi_regression_df.RT>0.3].reset_index(drop=True)
    roi_regression_df  = roi_regression_df[roi_regression_df.Round != 76].reset_index(drop=True)
    roi_regression_df  = roi_regression_df[roi_regression_df.decisionCPE != 0]
    
    roi_regression_df['cpe']             = roi_regression_df.decisionCPE.copy()
    roi_regression_df['cpe_split']       = roi_regression_df['cpe'].apply(lambda x: 0 if x < 0 else 1)
    roi_regression_df['TrialEV']         = roi_regression_df.GambleEV - roi_regression_df.SafeBet
    roi_regression_df['GambleChoice']    = roi_regression_df['GambleChoice'].astype('category')
    roi_regression_df['TrialType']       = roi_regression_df['TrialType'].astype('category')
    roi_regression_df['epoch_num']       = roi_regression_df.index # keep epoch as a column
    roi_regression_df['subj_id']         = subj_id
    roi_regression_df['roi']             = plot_roi
    roi_regression_df['bdi']             = subj_info_df[subj_info_df.subj_id == subj_id].bdi.values[0]
    roi_regression_df['unique_reref_ch'] = roi_regression_df[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)
    
    
#         # zscore continuous regressors AFTER removing bad trials!! 
    for reg in continuous_regressors:
        roi_regression_df[reg] = norm_zscore(roi_regression_df[reg].values) # zscore is /2std

    plot_dfs.append(roi_regression_df)
      

In [ ]:
roi_plot_df = pd.concat(plot_dfs).reset_index()

In [ ]:
roi_plot_df

# Start plotting

In [ ]:
sns.set_context('poster')
sns.set_style('ticks')



In [ ]:
power_avg.power_mean

In [ ]:
fig_name

In [ ]:
fig_dir   = f'{base_dir}ephys_analysis/figs/band_timeseries/{plot_band}/'
os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']

roi_plot_df['bdi_split'] = roi_plot_df['bdi'].apply(lambda x: 0 if x < 20 else 1)

sns.set_context('talk')

for bdi_thresh in roi_plot_df.bdi_split.unique():
    bdi_data    = roi_plot_df[roi_plot_df.bdi_split == bdi_thresh].reset_index(drop=True)    
    
    bdi_data['new_pow'] = np.nan
    if bdi_thresh == 0:
        bdi = 'low'
    else:
        bdi = 'high'

    for elec in bdi_data.unique_reref_ch.unique().tolist():
        elec_df = bdi_data[bdi_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        bdi_data['new_pow'][bdi_data.unique_reref_ch == elec] = elec_df['power']

        
    power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{bdi}BDI_{plot_roi}_{plot_band}.pdf'
    
    fig,ax    = plt.subplots(1,1,figsize=(6,4),dpi=400)
    plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
#     plt.ylim([np.round(np.min(power_avg.power_mean),3),np.round(np.max(power_avg.power_mean),3)])
#     plt.ylim([np.round(np.min(power_avg.power_mean),2),np.round(np.abs(np.max(power_avg.power_mean)),2)])
    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
#         plt.ylim([np.round(np.min(power_avg.power_mean),2),np.round(np.abs(np.max(power_avg.power_mean)),2)])

        
        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='x-small',shadow=None,markerscale=4,handlelength=1)

        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)


    fig.tight_layout()
    fig.suptitle(f'{bdi}BDI {plot_roi} {plot_band}')
#     ax.set_title(f'{bdi}BDI {plot_roi} {plot_band}',fontsize=20)
    plt.xlabel(r'Time (ms)')
    
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    
    sns.despine()
    
    plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
    bbox_inches='tight', pad_inches=0.1,
    facecolor='auto', edgecolor='auto',
    backend=None)



In [ ]:
fig_dir   = f'{base_dir}ephys_analysis/figs/roi_band_timeseries/{plot_roi}_{plot_band}/'
os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']

roi_plot_df['bdi_split'] = roi_plot_df['bdi'].apply(lambda x: 0 if x < 20 else 1)

for bdi_thresh in roi_plot_df.bdi_split.unique():
    bdi_data    = roi_plot_df[roi_plot_df.bdi_split == bdi_thresh].reset_index(drop=True)    
    
    bdi_data['new_pow'] = np.nan
    if bdi_thresh == 0:
        bdi = 'low'
    else:
        bdi = 'high'

    for elec in bdi_data.unique_reref_ch.unique().tolist():
        elec_df = bdi_data[bdi_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        bdi_data['new_pow'][bdi_data.unique_reref_ch == elec] = elec_df['power']

        
    power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{bdi}BDI_{plot_roi}_{plot_band}.pdf'
    
    fig,ax    = plt.subplots(1,1,figsize=(6,5),dpi=400)
    plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
    
    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)

        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)


#     plt.tight_layout()
    plt.title(f'{plot_roi} {plot_band} {elec} {bdi}BDI')
    plt.xlabel(r'Time (ms)')
    plt.ylim([np.round(np.min(power_avg),3),np.round(np.max(power_avg),3)])
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    sns.despine()

In [ ]:
##### plot by subj 

fig_dir   = f'{base_dir}ephys_analysis/figs/roi_band_timeseries/{plot_roi}_{plot_band}/'
os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']

roi_plot_df['bdi_split'] = roi_plot_df['bdi'].apply(lambda x: 0 if x < 20 else 1)

for bdi_thresh in roi_plot_df.bdi_split.unique():
    bdi_data    = roi_plot_df[roi_plot_df.bdi_split == bdi_thresh].reset_index(drop=True)    
    
    bdi_data['new_pow'] = np.nan
    if bdi_thresh == 0:
        bdi = 'low'
    else:
        bdi = 'high'

    for elec in bdi_data.unique_reref_ch.unique().tolist():
        elec_df = bdi_data[bdi_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        bdi_data['new_pow'][bdi_data.unique_reref_ch == elec] = elec_df['power']

        
    power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{bdi}BDI_{plot_roi}_{plot_band}.pdf'
    
    fig,ax    = plt.subplots(1,1,figsize=(6,5),dpi=400)
    plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
    
    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)

        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)


#     plt.tight_layout()
    plt.title(f'{plot_roi} {plot_band} {elec} {bdi}BDI')
    plt.xlabel(r'Time (ms)')
    plt.ylim([np.round(np.min(power_avg),3),np.round(np.max(power_avg),3)])
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    sns.despine()
    
#     plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
#     bbox_inches='tight', pad_inches=0.1,
#     facecolor='auto', edgecolor='auto',
#     backend=None)



In [ ]:
fig_dir   = f'{base_dir}ephys_analysis/figs/roi_band_timeseries/{plot_roi}_{plot_band}/'
os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']

unique_ch_list = roi_plot_df['unique_reref_ch'].unique().tolist()

for elec in unique_ch_list:
    elec_data    = roi_plot_df[roi_plot_df.unique_reref_ch == elec]
    subj_id      = elec_data.subj_id.unique()[0]
    subj_bdi     = elec_data.bdi.unique()[0]

    elec_data['power'] = elec_data.band_pow.rolling(window=500).mean()
    power_avg          = elec_data.groupby(['time','cpe_split']).agg(power_mean=('power', 'mean'), 
                                                    power_sem=('power', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{subj_id}_{elec}_{plot_roi}_{subj_bdi}.pdf'
    fig,ax    = plt.subplots(1,1,figsize=(6,4),dpi=400)
    plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)

    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])


        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)


        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)


    plt.tight_layout()
#         plt.title(f'{elec}')
    plt.xlabel(r'Time (ms)')
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    sns.despine()

    plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
    bbox_inches='tight', pad_inches=0.1,
    facecolor='auto', edgecolor='auto',
    backend=None)



In [ ]:
##### plot by subj 

fig_dir   = f'{base_dir}ephys_analysis/figs/roi_band_timeseries/{plot_roi}_{plot_band}/'

os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']

unique_subj_list = roi_plot_df['subj_id'].unique().tolist()

for subj_id in unique_subj_list:
    subj_data    = roi_plot_df[roi_plot_df.subj_id == subj_id].reset_index(drop=True)
    subj_bdi     = subj_data.bdi.unique()
    
    
    subj_data['new_pow'] = np.nan

    for elec in subj_data.unique_reref_ch.unique().tolist():
        elec_df = subj_data[subj_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        subj_data['new_pow'][subj_data.unique_reref_ch == elec] = elec_df['power']

        
    power_avg = subj_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{subj_id}_allelecs_{plot_roi}_{subj_bdi}.pdf'
    
    fig,ax    = plt.subplots(1,1,figsize=(6,4),dpi=400)
#     plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
    
    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)

        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)


    plt.tight_layout()
#         plt.title(f'{elec}')
    plt.xlabel(r'Time (ms)')
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    sns.despine()
    
    plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
    bbox_inches='tight', pad_inches=0.1,
    facecolor='auto', edgecolor='auto',
    backend=None)



In [ ]:
##### plot by subj 

fig_dir   = f'{base_dir}ephys_analysis/figs/roi_band_timeseries/{plot_roi}_{plot_band}/'
os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']

roi_plot_df['bdi_split'] = roi_plot_df['bdi'].apply(lambda x: 0 if x < 20 else 1)

for bdi_thresh in roi_plot_df.bdi_split.unique():
    bdi_data    = roi_plot_df[roi_plot_df.bdi_split == bdi_thresh].reset_index(drop=True)    
    
    bdi_data['new_pow'] = np.nan
    if bdi_thresh == 0:
        bdi = 'low'
    else:
        bdi = 'high'

    for elec in bdi_data.unique_reref_ch.unique().tolist():
        elec_df = bdi_data[bdi_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        bdi_data['new_pow'][bdi_data.unique_reref_ch == elec] = elec_df['power']

        
    power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{bdi}_all_{plot_roi}.pdf'
    
    fig,ax    = plt.subplots(1,1,figsize=(6,4),dpi=400)
    plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
    
    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)

        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)


    plt.tight_layout()
#         plt.title(f'{elec}')
    plt.xlabel(r'Time (ms)')
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    sns.despine()
    
    plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
    bbox_inches='tight', pad_inches=0.1,
    facecolor='auto', edgecolor='auto',
    backend=None)


In [ ]:
##### plot by subj 

fig_dir   = f'{base_dir}ephys_analysis/figs/roi_band_timeseries/{plot_roi}_{plot_band}/'
os.makedirs(fig_dir,exist_ok=True)
sem_colors = ['indianred','steelblue']
labels     = ['Negative CPE', 'Positive CPE']

roi_plot_df['bdi_split'] = roi_plot_df['bdi'].apply(lambda x: 0 if x < 20 else 1)

for bdi_thresh in roi_plot_df.bdi_split.unique():
    bdi_data    = roi_plot_df[roi_plot_df.bdi_split == bdi_thresh].reset_index(drop=True)    
    
    bdi_data['new_pow'] = np.nan
    if bdi_thresh == 0:
        bdi = 'low'
    else:
        bdi = 'high'

    for elec in bdi_data.unique_reref_ch.unique().tolist():
        elec_df = bdi_data[bdi_data.unique_reref_ch == elec]
        elec_df['power'] = elec_df.band_pow.rolling(window=500).mean()
        bdi_data['new_pow'][bdi_data.unique_reref_ch == elec] = elec_df['power']

        
    power_avg = bdi_data.groupby(['time','cpe_split']).agg(power_mean=('new_pow', 'mean'), 
                                                    power_sem=('new_pow', lambda x: np.std(x)/np.sqrt(len(x)))
                                                    ).reset_index()

    fig_name     = f'{fig_dir}{bdi}BDI_{plot_roi}_{plot_band}.pdf'
    
    fig,ax    = plt.subplots(1,1,figsize=(6,5),dpi=400)
    plt.axhline(0, color='black', linestyle='--',linewidth=2,alpha=0.7)
    
    # Plot the time resolved power with SEM shading
    for i, cpe_split in enumerate(power_avg['cpe_split'].unique()):

        subset = power_avg[power_avg['cpe_split'] == cpe_split]

        plt.plot(subset['time'], subset['power_mean'], label=labels[i],color=sem_colors[i])
        
        ax.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.5, 0.5,0,0.7),ncol=2,
                framealpha=0,fontsize='xx-small',shadow=None,markerscale=4,handlelength=1)

        plt.fill_between(subset['time'], subset['power_mean'] - subset['power_sem'], 
             subset['power_mean'] + subset['power_sem'], color=sem_colors[i], alpha=0.4)


#     plt.tight_layout()
    plt.title(f'{plot_roi} {plot_band} {elec} {bdi}BDI')
    plt.xlabel(r'Time (ms)')
    plt.ylabel(fr'${plot_band}_{{zpow}}$')
    sns.despine()
    
    plt.savefig(fig_name,dpi=400,format='pdf', metadata=None,
    bbox_inches='tight', pad_inches=0.1,
    facecolor='auto', edgecolor='auto',
    backend=None)

